In [2]:
#Mounting google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import pandas as pd
import numpy as np

# Importing Merged data
import_path = r"/content/gdrive/MyDrive/"
# export_path = r"C:\Users\admin\Desktop\Masters AI\Project A\Week 3"
# plots_path = r'C:\Users\admin\Desktop\Masters AI\Project A\Week 3\plots'


file, date_fields = 'weather_fire.csv', ['date','acq_date']
vars()[file[0:-4]] = pd.read_csv('/'.join([import_path,file[:]]),parse_dates=date_fields)
print(file,' = ',vars()[file[0:-4]].shape)

weather_fire.csv  =  (2054739, 31)


In [4]:
drop_features3 = ['date','acq_date','suburb','region','state','frp_min','frp_sum','frp_median','frp_mean','location','frp_max']

In [5]:
weather_fire['location']

0          Albany
1          Albany
2          Albany
3          Albany
4          Albany
            ...  
2054734     Young
2054735     Young
2054736     Young
2054737     Young
2054738     Young
Name: location, Length: 2054739, dtype: object

In [6]:
wf1b = weather_fire.copy()

wf1b['cat'] = 0

# # BINARY - frp_MAX 
wf1b['cat'] = np.where(wf1b['frp_max'] < 12, 0, wf1b['cat'])
wf1b['cat'] = np.where(wf1b['frp_max'] >= 12, 1, wf1b['cat'])

# print(wf1b.cat.value_counts(),'\n')

wf1b_ed = wf1b.drop(drop_features3, axis = 1) 
# We filter for Albany
# wf1_alb=wf1b_ed[wf1b_ed.location == 'Albany']
wf1_alb=wf1b_ed
# -----------------------------------------------------------------
# # Victoria only data
# wf1b_ed_vic = wf1b_ed[wf1b_ed.state_Victoria == 1]
# wf1b_ed_vic = wf1b_ed_vic.drop(states ,axis=1).reset_index(drop=True)
# -----------------------------------------------------------------

# DROP STATES
# wf1b_ed = wf1b_ed.drop(states ,axis=1).reset_index(drop=True)

pd.set_option('display.max_columns', 34)
wf1_alb

,maxC,minC,Snow_cm,sunHour,uv,DewPointC,FeelsLikeC,HeatIndexC,WindChillC,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,windAZI,windKmph,lat_suburb,long_suburb,cat
0,15,7,0.0,8.7,4,5,10,11,10,15,8,68,0.0,1023,15,10,183,9,-35.0031,117.86595,0
1,17,10,0.0,7.2,3,9,12,13,12,21,56,77,0.2,1017,17,10,177,13,-35.0031,117.86595,0
2,15,10,0.0,8.7,4,8,11,12,11,20,33,74,0.1,1023,15,10,180,13,-35.0031,117.86595,0
3,16,9,0.0,8.7,4,9,12,13,12,21,24,76,0.0,1022,16,10,82,13,-35.0031,117.86595,0
4,20,12,0.0,8.7,4,12,15,15,15,27,29,79,0.1,1012,20,10,37,17,-35.0031,117.86595,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2054734,28,15,0.0,14.5,6,14,23,23,22,11,6,64,0.0,1012,28,10,189,7,-34.3135,148.30107,0
2054735,29,15,0.0,14.5,7,15,24,24,23,9,4,66,0.0,1014,29,10,166,6,-34.3135,148.30107,0
2054736,28,20,0.0,13.4,7,16,25,25,24,11,25,61,0.8,1008,28,10,192,6,-34.3135,148.30107,0
2054737,25,20,0.0,10.2,5,15,23,23,22,19,61,69,8.4,1004,25,9,169,12,-34.3135,148.30107,0


In [7]:
def series_to_supervised(data,n_in=1,n_out=1,dropnan = True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols = list()
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
    for i in range(0, n_out):
        cols.append(df.shift(-i))
    agg = concat(cols, axis=1)
    if dropnan:
        agg.dropna(inplace=True)
    return agg.values

In [8]:
from sklearn.model_selection import train_test_split
# def walk_forward_validation(data, n_test):
#     predictions = list()
#     train, test = train_test_split(data, n_test)
#     history = [x for x in train]
#     for i in range(len(test)):
#         testX, testy = test[i, :-1], test[i, -1]
#         yhat = xgboost_forecast(history, testX)
#         predictions.append(yhat)
#         history.append(test[i])
#         print('>expected=%.1f, predicted=%.1f' % (testy, yhat))
#     error = mean_absolute_error(test[:, -1], predictions)
#     return error, test[:, 1], predictions
    

In [9]:
from numpy import asarray
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_absolute_error
from xgboost import XGBClassifier
from matplotlib import pyplot

In [10]:
# def xgboost_pred(train,testX):
#     train = asarray(train)
#     trainX, trainy = train[:, :-1], train[:, -1]
#     model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=1000)
#     model.fit(trainX, trainy)
#     yhat = model.predict([testX])
#     return yhat[0]

In [11]:
# X = wf1_alb.iloc[:,:-1]
# y = wf1_alb.iloc[:,-1]

In [12]:
dataset = wf1_alb
dataset = series_to_supervised(dataset)

In [13]:
dataset

array([[ 15.     ,   7.     ,   0.     , ..., -35.0031 , 117.86595,
          0.     ],
       [ 17.     ,  10.     ,   0.     , ..., -35.0031 , 117.86595,
          0.     ],
       [ 15.     ,  10.     ,   0.     , ..., -35.0031 , 117.86595,
          0.     ],
       ...,
       [ 29.     ,  15.     ,   0.     , ..., -34.3135 , 148.30107,
          0.     ],
       [ 28.     ,  20.     ,   0.     , ..., -34.3135 , 148.30107,
          0.     ],
       [ 25.     ,  20.     ,   0.     , ..., -34.3135 , 148.30107,
          0.     ]])

In [14]:
import pandas as pd
data = pd.DataFrame(dataset)
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,...,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
0,15.0,7.0,0.0,8.7,4.0,5.0,10.0,11.0,10.0,15.0,8.0,68.0,0.0,1023.0,15.0,10.0,183.0,...,3.0,9.0,12.0,13.0,12.0,21.0,56.0,77.0,0.2,1017.0,17.0,10.0,177.0,13.0,-35.0031,117.86595,0.0
1,17.0,10.0,0.0,7.2,3.0,9.0,12.0,13.0,12.0,21.0,56.0,77.0,0.2,1017.0,17.0,10.0,177.0,...,4.0,8.0,11.0,12.0,11.0,20.0,33.0,74.0,0.1,1023.0,15.0,10.0,180.0,13.0,-35.0031,117.86595,0.0
2,15.0,10.0,0.0,8.7,4.0,8.0,11.0,12.0,11.0,20.0,33.0,74.0,0.1,1023.0,15.0,10.0,180.0,...,4.0,9.0,12.0,13.0,12.0,21.0,24.0,76.0,0.0,1022.0,16.0,10.0,82.0,13.0,-35.0031,117.86595,0.0
3,16.0,9.0,0.0,8.7,4.0,9.0,12.0,13.0,12.0,21.0,24.0,76.0,0.0,1022.0,16.0,10.0,82.0,...,4.0,12.0,15.0,15.0,15.0,27.0,29.0,79.0,0.1,1012.0,20.0,10.0,37.0,17.0,-35.0031,117.86595,0.0
4,20.0,12.0,0.0,8.7,4.0,12.0,15.0,15.0,15.0,27.0,29.0,79.0,0.1,1012.0,20.0,10.0,37.0,...,4.0,13.0,15.0,16.0,15.0,42.0,38.0,85.0,1.7,1002.0,20.0,10.0,326.0,26.0,-35.0031,117.86595,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2054733,25.0,12.0,0.0,14.1,5.0,13.0,19.0,20.0,19.0,9.0,11.0,72.0,0.0,1012.0,25.0,9.0,180.0,...,6.0,14.0,23.0,23.0,22.0,11.0,6.0,64.0,0.0,1012.0,28.0,10.0,189.0,7.0,-34.3135,148.30107,0.0
2054734,28.0,15.0,0.0,14.5,6.0,14.0,23.0,23.0,22.0,11.0,6.0,64.0,0.0,1012.0,28.0,10.0,189.0,...,7.0,15.0,24.0,24.0,23.0,9.0,4.0,66.0,0.0,1014.0,29.0,10.0,166.0,6.0,-34.3135,148.30107,0.0
2054735,29.0,15.0,0.0,14.5,7.0,15.0,24.0,24.0,23.0,9.0,4.0,66.0,0.0,1014.0,29.0,10.0,166.0,...,7.0,16.0,25.0,25.0,24.0,11.0,25.0,61.0,0.8,1008.0,28.0,10.0,192.0,6.0,-34.3135,148.30107,0.0
2054736,28.0,20.0,0.0,13.4,7.0,16.0,25.0,25.0,24.0,11.0,25.0,61.0,0.8,1008.0,28.0,10.0,192.0,...,5.0,15.0,23.0,23.0,22.0,19.0,61.0,69.0,8.4,1004.0,25.0,9.0,169.0,12.0,-34.3135,148.30107,0.0


In [15]:
X = dataset[:,:-1]
y = dataset[:,-1]

In [16]:
len(X)

2054738

In [17]:
len(y)

2054738

In [18]:
y

array([0., 0., 0., ..., 0., 0., 0.])

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [20]:
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

undersample = RandomUnderSampler(sampling_strategy='majority', random_state=24)
x_under, y_under = undersample.fit_resample(X_train, y_train)
print('')
print(Counter(y_train))
print(Counter(y_under))

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)



Counter({0.0: 1390114, 1.0: 48202})
Counter({0.0: 48202, 1.0: 48202})


In [21]:
# import numpy as np
# X_train = asarray(X_train)

In [55]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]


# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [56]:
# Use the random grid to search for best hyperparameters
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
scoring = {'Recall': 'recall', 'Accuracy': make_scorer(accuracy_score)}
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,scoring= scoring, n_iter = 100, cv = 2, verbose=2, random_state=42, n_jobs = -1,refit='Recall')

# Fit the grid search model
rf_random.fit(x_under, y_under)

Fitting 2 folds for each of 100 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 38.6min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 174.9min
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed: 215.0min finished


RandomizedSearchCV(cv=2, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [57]:
rf_random.best_params_
#rf_grid.cv_results_

# GridSearchCV(cv=3, error_score=nan,
#              estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
#                                               class_weight=None,
#                                               criterion='gini', max_depth=None,
#                                               max_features='auto',
#                                               max_leaf_nodes=None,
#                                               max_samples=None,
#                                               min_impurity_decrease=0.0,
#                                               min_impurity_split=None,
#                                               min_samples_leaf=1,
#                                               min_samples_split=2,
#                                               min_weight_fraction_leaf=0.0,
#                                               n_estimators=100, n_jobs=None,
#                                               oob_score=False,
#                                               random_state=None, verbose=0,
#                                               warm_start=False)


{'bootstrap': False,
 'max_depth': 90,
 'max_features': 'auto',
 'min_samples_leaf': 2,
 'min_samples_split': 5,
 'n_estimators': 200}

In [58]:
modelRF = RandomForestClassifier(bootstrap=False, max_depth=90,max_features='auto',min_samples_leaf=2, min_samples_split=5, n_estimators=200,random_state=42)

modelRF.fit(x_under,y_under)

RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=90, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [59]:
y_pred = modelRF.predict(X_test)

In [47]:
y_pred

array([0., 0., 0., ..., 0., 0., 0.])

In [60]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.8976658847348084

In [61]:
from sklearn.metrics import classification_report
clr = classification_report(y_test, y_pred)

In [62]:
print(clr)

              precision    recall  f1-score   support

         0.0       0.99      0.90      0.94    595783
         1.0       0.23      0.86      0.36     20639

    accuracy                           0.90    616422
   macro avg       0.61      0.88      0.65    616422
weighted avg       0.97      0.90      0.92    616422



In [44]:
from sklearn.metrics import confusion_matrix
cf = confusion_matrix(y_test, y_pred)
print(cf)

[[536536  59247]
 [  3320  17319]]
